## Imports

In [1]:
# | code-fold: true
# | code-summary: "Load packages"
# | output: false


import os
import numpy as np
import os
import numpy as np
from sympy import Matrix, sqrt, Piecewise
import sympy as sp
import pytest
from attr import define, field
from sympy import MutableDenseNDimArray as Arr


from zoomy_core.fvm.solver_numpy import Settings
from zoomy_core.model.basemodel import Model
import zoomy_core.model.initial_conditions as IC
import zoomy_core.model.boundary_conditions as BC
from zoomy_core.misc.misc import Zstruct, ZArray
import zoomy_core.misc.misc as misc
import zoomy_firedrake.firedrake_solver as dg


In [2]:
@define(frozen=True, slots=True, kw_only=True)
class SWE(Model):
    dimension: int = 2
    variables: Zstruct = field(init=False)
    aux_variables: Zstruct = field(default=1)
    _default_parameters: dict = field(
        init=False, factory=lambda: {"g": 9.81, "ex": 0.0, "ey": 0.0, "ez": 1.0, "rho": 1000.0, "n": 0.01, "eps":1e-4}
    )
    
    def __attrs_post_init__(self):
        object.__setattr__(self, "variables", self.dimension + 2)
        super().__attrs_post_init__()

    def project_2d_to_3d(self):
        out = ZArray.zeros(6)
        p = self.parameters
        dim = self.dimension
        z = self.position[2]
        b = self.aux_variables[0]
        h = self.variables[1]
        U = [hu / h for hu in self.variables[2 : 2 + dim]]
        out[0] = b
        out[1] = h
        out[2] = U[0]
        out[3] = 0 if dim == 1 else U[1]
        out[4] = 0
        out[5] = p.rho * p.g * h * (1 - z)
        return out
    
    def get_primitives(self):
        dim = self.dimension
        b = self.variables[0]
        h = self.variables[1]
        hinv = 1/h
        U = Matrix([hu * hinv for hu in self.variables[2 : 2 + dim]])
        return b, h, U, hinv

    def flux(self):
        dim = self.dimension
        b, h, U, hinv = self.get_primitives()
        g = self.parameters.g
        I = Matrix.eye(dim)
        F = Matrix.zeros(self.variables.length(), dim)
        F[1, :] = h * U.T
        # F[2:, :] = h * U * U.T + g / 2 * h**2 * I
        F[2:, :] = h * U * U.T
        return ZArray(F)
    
    def nonconservative_matrix(self):
        dim = self.dimension
        b, h, U, hinv = self.get_primitives()
        U = Matrix([hu * hinv for hu in self.variables[2 : 2 + dim]])
        g = self.parameters.g
        N = ZArray.zeros(self.n_variables, self.n_variables, dim)
        for d in range(dim):
            N[2+d, 0, d] = g * h # g * h * grad(b)
            N[2+d, 1, d] = g * h # g * h * grad(h)
        return ZArray(N)
    
    def source(self):
        eps = 1e-16
        dim = self.dimension
        _, _, U, _ = self.get_primitives()
        hinv = self.aux_variables[0]
        # Uold = Matrix(self.aux_variables[1 : 1 + dim])
        g = self.parameters.g
        n = self.parameters.n
        abs_u = sqrt(U.dot(U) + eps)
        S = Matrix.zeros(self.n_variables, 1)
        # S[2:, 0] = -n**2 * g * hinv**(1/3) * U[:, 0] * abs_u
        S[2:, 0] = n**2 * g  * (hinv**(1/3) + eps) * U[:, 0] * abs_u
        return ZArray(S).reshape(self.n_variables,)
    
    # def left_eigenvectors(self):
    #     A = self.normal[0] * self.quasilinear_matrix()[:, :, 0]
    #     for d in range(1, self.dimension):
    #         A += self.normal[d] * self.quasilinear_matrix()[:, :, d]
    #     D, P = sp.Matrix(A).diagonalize()
    #     L = P**(-1)
    #     return self._simplify(L)
    
    # def right_eigenvectors(self):
    #     A = self.normal[0] * self.quasilinear_matrix()[:, :, 0]
    #     for d in range(1, self.dimension):
    #         A += self.normal[d] * self.quasilinear_matrix()[:, :, d]
    #     D, P = sp.Matrix(A).diagonalize()
    #     R = P
    #     return self._simplify(R)
    


@define(frozen=True, slots=True, kw_only=True)
class NumericSWE(SWE):
    disable_differentiation: bool = True
    
    def get_primitives(self):
        dim = self.dimension
        b = self.variables[0]
        h = self.variables[1]
        hinv = self.aux_variables[0]
        U = Matrix([hu * hinv for hu in self.variables[2 : 2 + dim]])
        return b, h, U, hinv
    
    def eigenvalues(self):
        ev = super().eigenvalues()
        h = self.variables[1]
        return sp.Function('conditional')(h > self.parameters.eps, ev, ZArray.zeros(*ev.shape))
    
    def source(self):
        delta = self.parameters.eps  # or smaller
        h = self.variables[1]
        smooth = sp.Rational(1,2)*(1 + sp.tanh((h - self.parameters.eps)/delta))

        S = super().source()
        zeros = ZArray.zeros(*S.shape)

        return smooth * S + (1 - smooth) * zeros
    
    def left_eigenvectors(self):
        return ZArray.zeros(self.n_variables, self.n_variables)


    def right_eigenvectors(self):
        return ZArray.zeros(self.n_variables, self.n_variables)
                



# Transformation to UFL Code (Medium)

### Map from Sympy to UFL

In [3]:
swe = SWE(dimension=1)
P, D = sp.Matrix(swe.quasilinear_matrix()[1:,1:,0]).diagonalize()
R = sp.simplify(P)
D = sp.simplify(D)
L = sp.simplify(P**-1)
sp.simplify(R @ D @ L)

⎡    0        1  ⎤
⎢                ⎥
⎢         2      ⎥
⎢       q₂   2⋅q₂⎥
⎢g⋅q₁ - ───  ────⎥
⎢         2   q₁ ⎥
⎣       q₁       ⎦

In [4]:
R2 = sp.eye(4)

R2[2:, 2:] = R
L2 = R2.inv()
D2 = sp.diag(0, *swe.eigenvalues())

In [5]:
A_approx = sp.simplify(R2 @ D2 @ L2)
A_approx

⎡0  0         0             0   ⎤
⎢                               ⎥
⎢0  0         0             0   ⎥
⎢                               ⎥
⎢0  0         0            n₀   ⎥
⎢                               ⎥
⎢         ⎛    3     2⎞         ⎥
⎢      n₀⋅⎝g⋅q₁  - q₂ ⎠  2⋅n₀⋅q₂⎥
⎢0  0  ────────────────  ───────⎥
⎢              2           q₁   ⎥
⎣            q₁                 ⎦

In [6]:
swe = SWE()
S = sp.eye(4)
S[2, 2] =  swe.normal[0]
S[3, 2] = -swe.normal[1]
S[2, 3] = swe.normal[1]
S[3, 3] = swe.normal[0]
Sinv = S.inv()
Q = ZArray(swe.variables.get_list())
An = sp.simplify(sum((sp.Matrix(swe.quasilinear_matrix()[:, :, di]) * swe.normal[di] for di in range(2)), start=sp.Matrix.zeros(4,4)))
Anr = sp.simplify(S @ An @ Sinv)
Anr

⎡       0                                           0                          ↪
⎢                                                                              ↪
⎢       0                                           0                          ↪
⎢                                                                              ↪
⎢                     ⎛   ⎛    3     2⎞           ⎞      ⎛              ⎛    3 ↪
⎢     ⎛  2     2⎞  n₀⋅⎝n₀⋅⎝g⋅q₁  - q₂ ⎠ - n₁⋅q₂⋅q₃⎠ - n₁⋅⎝n₀⋅q₂⋅q₃ - n₁⋅⎝g⋅q₁  ↪
⎢g⋅q₁⋅⎝n₀  + n₁ ⎠  ─────────────────────────────────────────────────────────── ↪
⎢                                                    2                         ↪
⎢                                                  q₁                          ↪
⎢                                                                              ↪
⎢                                2                 2           2     2         ↪
⎢                            - n₀ ⋅q₂⋅q₃ + n₀⋅n₁⋅q₂  - n₀⋅n₁⋅q₃  + n₁ ⋅q₂⋅q₃   ↪
⎢       0                   

In [7]:
An

⎡   0                  0                       0                0       ⎤
⎢                                                                       ⎥
⎢   0                  0                      n₀               n₁       ⎥
⎢                                                                       ⎥
⎢            ⎛    3     2⎞                                              ⎥
⎢         n₀⋅⎝g⋅q₁  - q₂ ⎠ - n₁⋅q₂⋅q₃   2⋅n₀⋅q₂ + n₁⋅q₃       n₁⋅q₂     ⎥
⎢g⋅n₀⋅q₁  ───────────────────────────   ───────────────       ─────     ⎥
⎢                       2                     q₁               q₁       ⎥
⎢                     q₁                                                ⎥
⎢                                                                       ⎥
⎢                        ⎛    3     2⎞                                  ⎥
⎢         -n₀⋅q₂⋅q₃ + n₁⋅⎝g⋅q₁  - q₃ ⎠       n₀⋅q₃       n₀⋅q₂ + 2⋅n₁⋅q₃⎥
⎢g⋅n₁⋅q₁  ────────────────────────────       ─────       ───────────────⎥
⎢                       2             

In [8]:
bcs = BC.BoundaryConditions(
    [
        BC.Extrapolation(tag="wall"),
        BC.Extrapolation(tag="inflow"),
        BC.Extrapolation(tag="outflow"),
        # BC.Wall(tag="wall", momentum_field_indices=[[2, 3]], wall_slip=0.8),
        # BC.Wall(tag="inflow", momentum_field_indices=[[2, 3]], wall_slip=0.8),
        # BC.Wall(tag="outflow", momentum_field_indices=[[2, 3]], wall_slip=0.8),
    ]
)

 ### Initial condition
def ic_q(x):
    R = 3
    r = np.sqrt((x[0])**2 + (x[1])**2)
    # b = 0.1 * x[0] + 0.5 * np.sin(2 * np.pi * x[0] / 5)
    b = 0.3 * x[0]
    # b = 0
    h = np.where(r <= R, 2., 1) -b
    h = np.where(h <= 0, 0, h)
    return np.array([b, h , 0.*x[0], 0.*x[0]])

ic = IC.UserFunction(ic_q)

model = NumericSWE(
    dimension=2,
    boundary_conditions=bcs,
    initial_conditions=ic,
)

settings = Settings(name="Firedrake", output=Zstruct(directory="outputs/firedrake", snapshots=1000, filename='dg', clean_directory=True))


In [9]:
import ufl 
IdentityMatrix = ufl.as_tensor([[0, 0, 0, 0], [1, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

solver = dg.FiredrakeHyperbolicSolver(settings=settings, time_end = 10.0, CFL=0.5, IdentityMatrix=IdentityMatrix)

In [10]:
main_dir = misc.get_main_directory()
path_to_mesh = os.path.join(main_dir, "meshes", "square", "mesh.msh")
solver.solve(path_to_mesh, model)

2025-11-14 05:36:05.708 | INFO     | zoomy_firedrake.firedrake_solver:solve:431 - iteration: 10, time: 0.142536, dt: 0.014312, next write at time: 0.110110
2025-11-14 05:36:06.388 | INFO     | zoomy_firedrake.firedrake_solver:solve:431 - iteration: 20, time: 0.286153, dt: 0.014398, next write at time: 0.210210
2025-11-14 05:36:07.071 | INFO     | zoomy_firedrake.firedrake_solver:solve:431 - iteration: 30, time: 0.430508, dt: 0.014464, next write at time: 0.310310
2025-11-14 05:36:07.761 | INFO     | zoomy_firedrake.firedrake_solver:solve:431 - iteration: 40, time: 0.575470, dt: 0.014521, next write at time: 0.410410
2025-11-14 05:36:08.527 | INFO     | zoomy_firedrake.firedrake_solver:solve:431 - iteration: 50, time: 0.720962, dt: 0.014571, next write at time: 0.510511
2025-11-14 05:36:09.252 | INFO     | zoomy_firedrake.firedrake_solver:solve:431 - iteration: 60, time: 0.866928, dt: 0.014616, next write at time: 0.610611
2025-11-14 05:36:09.923 | INFO     | zoomy_firedrake.firedrake_s